In [5]:
import os
import numpy as np
import pandas as pd

from astropy.modeling import blackbody as bb
from astropy.table import Table, Column, Row
from astropy import units as u

from lmfit import minimize, Parameters, fit_report

import pycoco as pcc
%matplotlib inline

# Extend LC

In [26]:
path = pcc.defaults._default_data_dir_path

files = os.listdir(path + '/lc_raw/')
files.remove('README')

for lc in files:
    print(lc)

    data = pd.read_csv(path + '/lc_raw/'+lc, delim_whitespace=True, header=None, comment='#')
    data.columns = ['mjd', 'flux', 'flux_err', 'band']
    V = data.query('band == "BessellV"')

    if V.size > 0:
        V['flux'] /= 10
        V['flux_err'] /= 10
        V['band'] = "swift_UVW1"

        data = pd.concat((data, V))
        data.to_csv(path + 'lc_extend/' + lc, header=False, sep=' ', index=False)

SN1996cb.dat


/Users/szymon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/szymon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/szymon/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

SN2011dh.dat
SN2012ap.dat
SN2007uy.dat
SN2013ge.dat
SN1993J.dat
SN2011fu.dat
SN2005bf.dat
iPTF13bvn.dat
SN1999dn.dat
SN2011ei.dat
SN2009iz.dat
SN2005hg.dat
SN2006aj.dat
SN2011bm.dat
SN1994I.dat
SN2009jf.dat
SN2008bo.dat
SN2007C.dat
SN2007Y.dat
SN2008ax.dat
SN2002ap.dat
SN2004aw.dat
SN2009er.dat
SN2003jd.dat
SN2007gr.dat
SN2008D.dat
SN1998bw.dat
SN2010al.dat
SN2009bb.dat


In [27]:
for file in os.listdir(path + 'lc_extend/'):
    os.system("echo '# MJD flux flux_err filter' | cat - {0} > {1}".format(path + 'lc_extend/' + file,
                                                                       path + 'lc/' + file))

# Spectrum Extensions

In [5]:
params = Parameters()
params.add("T", value=10000, min=1000) 
params.add("flux_scale", value=1e23)
params.add("EBV", value=0.001, min=0.0) 


def bb_min_fun(params, filters, wavelength, verbose=False):
    T = params["T"]
    flux_scale = params["flux_scale"]
    EBV = params["EBV"]
    
    bb_flux = bb.blackbody_lambda(np.array(wavelength)*u.AA, temperature=T*u.Kelvin)

    bb_spec = pcc.classes.SpectrumClass()
    bb_spec.load_table(Table([wavelength, bb_flux], names=("wavelength", "flux")))

    bb_spec.flux = pcc.extinction.unred(bb_spec.wavelength, bb_spec.flux, EBV=EBV)

    bb_spec.data["flux"] = bb_spec.data["flux"] / flux_scale
    bb_spec.flux = bb_spec.flux / flux_scale

    bb_spec.get_specphot(filters, verbose=False)
    
    return bb_spec


def bb_min(params, specphot, filters, wavelength, verbose=False):
    bb_flux = bb_min_fun(params, filters, wavelength, verbose).specphot["flux"]

    residual = specphot["flux"] - bb_flux
    residual = residual[~np.isnan(residual)]

    return residual

In [6]:
for snname in os.listdir(path + '/spec/'):
    if not os.path.exists(path + 'spec_extend/' + snname):
        os.mkdir(path + 'spec_extend/' + snname)
    
    sn = pcc.classes.SNClass(snname)

    sn.load_phot(path = os.path.join(pcc.defaults._default_data_dir_path, "lc/" + snname + ".dat"))
    sn.load_list(os.path.join(pcc.defaults._default_list_dir_path, snname + ".list"))
    sn.load_spec()
    sn.check_overlaps()

    for file_name, Spec in sn.spec.items():
        if os.path.exists(path + 'spec_extend/' + snname + '/' + file_name):
            print('Skipping', file_name)
            continue
        print(file_name)

        Spec.specphot = Table(names=("lambda_effective", "flux", "filter"), dtype=('f4', 'f4', 'S'))
        for filter_name in sn.phot.data_filters:
            flux = pcc.kcorr.calc_spectrum_filter_flux(filter_object=sn.phot.data_filters[filter_name], 
                                                       spectrum_object=Spec, verbose=False)

            Spec.specphot.add_row((sn.phot.data_filters[filter_name].lambda_effective, 
                                   flux, 
                                   filter_name))
            wav = sn.phot.data_filters[filter_name].wavelength
        
        out = minimize(bb_min, 
                       params, 
                       args=(Spec.specphot, sn.phot.data_filters, wav))
        best_bb = bb_min_fun(out.params, 
                             sn.phot.data_filters, 
                             Column(np.arange(2000, 11000)*u.Angstrom))
        
        w_red = np.where(best_bb.wavelength > np.nanmax(Spec.wavelength))
        
        idx = np.where(Spec.wavelength < np.min(Spec.wavelength) + 50)

        low = 0.2 * np.mean(Spec.flux)
        w_blue = np.arange(1500, np.min(Spec.wavelength))
        flux_blue = np.linspace(low, np.mean(Spec.flux[idx]), w_blue.size)
        
        new_flux = np.concatenate((flux_blue, Spec.flux, best_bb.flux[w_red]))
        new_wav = np.concatenate((w_blue, Spec.wavelength, best_bb.wavelength[w_red]))
        
        df = pd.DataFrame({'wav': new_wav, 'flux': new_flux})
        df.to_csv(path + 'spec_extend/' + snname + '/' + file_name, 
                  columns=['wav', 'flux'],
                  header=False, 
                  sep=' ', 
                  index=False)

Skipping 2009bb_-4.85.txt
Skipping 2009bb_-3.86.txt
Skipping 2009bb_1.09.txt
Skipping 2009bb_5.05.txt
Skipping 2009bb_14.95.txt
Skipping 2009bb_15.94.txt
Skipping 2009bb_19.9.txt
Skipping 2009bb_20.89.txt
Skipping 2009bb_23.86.txt
Skipping 2009bb_27.82.txt
Skipping 2005hg_-12.54.txt
Skipping 2005hg_-11.56.txt
Skipping 2005hg_-9.6.txt
Skipping 2005hg_-8.62.txt
Skipping 2005hg_-7.64.txt
Skipping 2005hg_-6.66.txt
Skipping 2005hg_-5.68.txt
Skipping 2005hg_-4.7.txt
Skipping 2005hg_-3.72.txt
Skipping 2005hg_-2.74.txt
Skipping 2005hg_-1.76.txt
Skipping 2005hg_-0.78.txt
Skipping 2005hg_0.2.txt
Skipping 2005hg_12.93.txt
Skipping 2005hg_16.85.txt
Skipping 2005hg_22.72.txt
Skipping 2005hg_26.64.txt
Skipping 2007gr_-10.18.txt
Skipping 2007gr_-9.18.txt
Skipping 2007gr_-7.19.txt
Skipping 2007gr_-6.19.txt
Skipping 2007gr_-5.19.txt
Skipping 2007gr_-4.19.txt
Skipping 2007gr_-1.2.txt
Skipping 2007gr_3.79.txt
Skipping 2007gr_5.79.txt
Skipping 2007gr_6.79.txt
Skipping 2007gr_7.79.txt
Skipping 2007gr_8.79.

/Users/szymon/anaconda/lib/python3.5/site-packages/pycocosn-0.9.19-py3.5.egg/pycoco/kcorr.py:247: RuntimeWarning: invalid value encountered in double_scalars
  return  integrated_flux/filter_area
/Users/szymon/anaconda/lib/python3.5/site-packages/pycocosn-0.9.19-py3.5.egg/pycoco/classes.py:629: RuntimeWarning: divide by zero encountered in double_scalars
  flux = integrated_flux / filter_area


2012ap_2.07.txt
2012ap_4.05.txt
2012ap_7.02.txt
2012ap_8.0.txt
2012ap_13.93.txt
2012ap_24.8.txt
2009er_-5.12.txt
2009er_-4.15.txt
2009er_-2.22.txt
2009er_-1.26.txt
2009er_0.68.txt
2009er_2.61.txt
2009er_15.17.txt
2009er_16.14.txt
iPTF13bvn_-15.73.txt
iPTF13bvn_-14.73.txt
iPTF13bvn_-13.74.txt
iPTF13bvn_-12.74.txt
iPTF13bvn_-10.75.txt
iPTF13bvn_-8.76.txt
iPTF13bvn_-7.77.txt
iPTF13bvn_-6.77.txt
iPTF13bvn_-5.77.txt
iPTF13bvn_-4.78.txt
iPTF13bvn_-1.79.txt
iPTF13bvn_1.19.txt
iPTF13bvn_8.16.txt
iPTF13bvn_20.11.txt
iPTF13bvn_34.05.txt
2003jd_-3.14.txt
2003jd_-2.16.txt
2003jd_-1.18.txt
2003jd_-0.2.txt
2003jd_0.79.txt
2003jd_3.73.txt
2003jd_14.53.txt
2003jd_17.47.txt
2003jd_18.45.txt
2003jd_19.43.txt
2003jd_20.42.txt
2003jd_21.4.txt
2003jd_23.36.txt
2003jd_26.31.txt
2003jd_27.29.txt
2003jd_28.27.txt
2003jd_33.18.txt
2003jd_45.94.txt
2003jd_46.92.txt
2003jd_47.9.txt
2003jd_48.88.txt


In [17]:
for snname in os.listdir(path + 'spec_extend/'):
    os.mkdir(path + 'spec/' + snname)
    for file in os.listdir(path + 'spec_extend/' + snname):
        extend = path + 'spec_extend/' + snname + '/' + file
        spec = path + 'spec/' + snname + '/' + file
        os.system("echo '# wavelength flux' | cat - {0} > {1}".format(extend, spec))

In [16]:
spec

'/Users/szymon/Projects/CoCo/data/spec/SN2003jd/2003jd_23.36.txt'